In [9]:
from imports import *
from functions import *
from constants import *

In [2]:
try:
    driver = create_driver()
    wait = WebDriverWait(driver, timeout=10)
    # Suppression de l'élément Cambiar Pais
    driver.get(website_url)
    cambiarPais = wait.until(EC.presence_of_all_elements_located((By.XPATH,
                        "//div[contains(@id, 'cambiarPais')]")))
    driver.execute_script("""
    var element = document.querySelector("#cambiarPais");
    if (element) {
        element.remove();  // Cette ligne supprime l'élément du DOM
    }
""")
    driver.refresh()
    wait = WebDriverWait(driver, 10)
except :
    pass

# Connection to Account

In [3]:
try:
    driver.get(website_url)

    # Find Mi Cuenta and click on it
    mi_cuenta = wait.until(EC.element_to_be_clickable((By.XPATH,
        "//a[contains(@title, 'Mi Cuenta')]")))
    if not mi_cuenta.is_selected():
        mi_cuenta.click()
       
except Exception as e:
    print(str(e))

try:
    email_input = wait.until(EC.presence_of_element_located((By.XPATH, 
                    "//input[contains(@id, 'signin_username')]")))
    if email_input:
        print("Email Input found")
        email_input.send_keys(my_email)
        email_input.send_keys(Keys.ENTER)

except Exception as e:
    print(str(e)) 

try:
    password_input = wait.until(EC.presence_of_element_located((By.XPATH, 
                    "//input[contains(@id, 'signin_password')]")))
    if password_input:
        print("Password Input found")
        password_input.send_keys(my_password)
        password_input.send_keys(Keys.ENTER)

except Exception as e:
    print(str(e)) 


Email Input found
Password Input found


# Queries

In [4]:
QUERY_CATEGORY = "technology"
QUERY_SEARCH = ""
QUERY_BOOK_STATE = ["new"]
QUERY_BOOK_FORMAT = "paper"

In [5]:
driver.get(website_url)

try:
    # 📦 Récupérer le menu contenant les catégories
    categories_menu = find_element(driver, "xpath", "//div[contains(@class, 'categories')]")
    print("✅ Categories menu found!")

    # 📚 Récupérer toutes les catégories disponibles dans ce menu
    categories = find_elements(categories_menu, "xpath",
                               ".//li[contains(@class, 'category-li') and contains(@itemprop,'')]")
    print(f"✅ {len(categories)} categories found!")

    # 🔍 Préparation de la recherche
    if QUERY_CATEGORY:
        query_category = process_string(QUERY_CATEGORY).strip().lower()
        print(f"🔍 Searching for category: '{query_category}'")

        # 🔁 Parcourir les catégories et chercher une correspondance
        for cat in categories:
            category_name = cat.text.strip().lower()
            try:
                link = find_element(cat, "xpath", ".//a[contains(@itemprop, 'WPSideBar')]")
                category_href = link.get_attribute("href")
            except Exception as e:
                print(f"❌ Failed to get link for category: {e}")
                continue

            print(f"🔄 Comparing:\n→ Query: '{query_category}'\n→ Category: '{category_name}'")
            print(f"🔗 Link: {category_href}\n")

            if query_category in category_name:
                print(f"✅ Match found! Navigating to: {category_href}")
                driver.get(category_href)
                break  # Arrêter la boucle dès qu'une catégorie correspondante est trouvée

    # 🧭 Page de résultats après navigation
    page_results = driver.current_url
    print(f"📄 Current page: {page_results}")

except Exception as e:
    print(f"❌ Unexpected error occurred:\n{e}")


✅ Categories menu found!
✅ 25 categories found!
🔍 Searching for category: 'tecnología'
🔄 Comparing:
→ Query: 'tecnología'
→ Category: 'artes'
🔗 Link: https://www.buscalibre.com.co/libros/arte

🔄 Comparing:
→ Query: 'tecnología'
→ Category: 'biografías, literatura y estudios literarios'
🔗 Link: https://www.buscalibre.com.co/libros/filologia

🔄 Comparing:
→ Query: 'tecnología'
→ Category: 'ciencias de la tierra, geografía, medioambiente, planificación'
🔗 Link: https://www.buscalibre.com.co/libros/tierra-medio-ambiente

🔄 Comparing:
→ Query: 'tecnología'
→ Category: 'computación y tecnología de la información'
🔗 Link: https://www.buscalibre.com.co/libros/computacion

✅ Match found! Navigating to: https://www.buscalibre.com.co/libros/computacion
📄 Current page: https://www.buscalibre.com.co/libros/computacion


Filters Choice

In [11]:
try:
    print("🔍 Traitement de QUERY_BOOK_STATE...")
    current_url = driver.current_url.split("?")[0]

    if all(x in QUERY_BOOK_STATE for x in ["new", "used"]):
        url_new_and_used_books = current_url + "?condition=nuevosyusados"
        print(f"📚 Redirection vers : {url_new_and_used_books}")
        driver.get(url_new_and_used_books)

    elif any("new" in q.lower() for q in QUERY_BOOK_STATE):
        url_new_books = current_url + "?condition=new"
        print(f"📘 Redirection vers livres neufs : {url_new_books}")
        driver.get(url_new_books)

    elif any("used" in q.lower() for q in QUERY_BOOK_STATE):
        url_used_books = current_url + "?condition=used"
        print(f"📕 Redirection vers livres d'occasion : {url_used_books}")
        driver.get(url_used_books)
    else:
        print("ℹ️ Aucun état de livre applicable trouvé.")

except Exception as e:
    print(f"❌ Erreur dans le traitement de QUERY_BOOK_STATE : {e}")

# ------------------------------

try:
    print("🔍 Traitement de QUERY_BOOK_FORMAT...")
    current_url = driver.current_url.split("?")[0]

    if QUERY_BOOK_FORMAT.lower() == "ebook":
        url_ebook = current_url + "?condition=ebook"
        print(f"📱 Redirection vers ebook : {url_ebook}")
        driver.get(url_ebook)

    elif "audio" in QUERY_BOOK_FORMAT.lower():
        url_audio = current_url + "?condition=audio"
        print(f"🎧 Redirection vers livre audio : {url_audio}")
        driver.get(url_audio)
    else:
        print("ℹ️ Aucun format spécifique trouvé.")

except Exception as e:
    print(f"❌ Erreur dans le traitement de QUERY_BOOK_FORMAT : {e}")


🔍 Traitement de QUERY_BOOK_STATE...
📘 Redirection vers livres neufs : https://www.buscalibre.com.co/libro-nexus/9789566247593/p/62355533?condition=new
🔍 Traitement de QUERY_BOOK_FORMAT...
ℹ️ Aucun format spécifique trouvé.


Page Details Scraping

In [ ]:
def get_book_details(driver):
    book_details = {}
    print("📄 Page chargée avec succès.")
    
    products_listings = find_element(driver, "xpath", "//div[contains(@class, 'productos')]")
    cards = find_elements(products_listings, "xpath", "//div[contains(@class, 'box-producto') and contains(@data-id_producto, '')]")
    card = cards[0]

    title = safe_get_text(card, ".//h3[contains(@class, 'nombre')]", "Title", book_details, "Title")
    href = safe_get_text(card, ".//a[contains(@href, '')]", "Link", book_details, "Link", attr="href")
    autor = safe_get_text(card, ".//div[contains(@class, 'autor')]", "Autor", book_details, "Author")
    edition = safe_get_text(card, ".//div[contains(@class, 'autor') and contains(@class,'color-dark-gray')]", "Edition", book_details, "Edition")
    

    price = safe_get_text(card, ".//p[contains(@class, 'precio-ahora')]", "Actual Price", book_details, "Price")

    try:
        precio_antes = find_element(card, "xpath", ".//p[contains(@class, 'precio-antes')]").text.strip()
        book_details["Old Price"] = precio_antes
        print(f"Old Price: {precio_antes}\n")
        match_digit_precio = re.search(r'\d+\.\d+', price)
        match_digit_precio_antes = re.search(r'\d+\.\d+', precio_antes)
        if match_digit_precio and match_digit_precio_antes:
            match_digit_precio = match_digit_precio.group()
            match_digit_precio_antes = match_digit_precio_antes.group()
        discount_value = 100 - int(100 * float(match_digit_precio) / float(match_digit_precio_antes))
        discount = f"{discount_value}%"
        book_details["Discount"] = discount
        print(f"Discount: {discount}")
    except:
        print("Old Price not found")
    print("Going to product page...")

    ####### GO ON PRODUCT PAGE ######################################################################################################################################################

    driver.get(href)
    wait_page_to_load(driver)
    print("📄 Page produit chargée avec succès.")

    try:
        info_libro = find_element(driver, "xpath", "//div[contains(@id, 'data-info-libro')]")
        book_year = safe_get_text(driver, 
            "//div[contains(@class, 'box') and contains(@id, 'metadata-ano')]", "year", book_details, "📅 Année de publication")
    except Exception as e:
        print(f"❌ Erreur lors de la récupération des informations du livre :\n{e}")


    try:
        rating_amount = find_element(driver, "xpath", "//a[contains(@class, 'valoracion')]").text
        match_rating_amount = re.search(r'\d+', rating_amount.replace('.', '').replace(',', ''))
        if match_rating_amount:
            rating_amount = match_rating_amount.group()
            book_details["rating_amount"] = rating_amount
        print(f"📊 Nombre d'avis : {rating_amount}")
    except Exception as e:
        print(f"❌ Erreur sur le nombre d'avis :\n{e}")

    try:
        stocks_details = safe_get_text(driver, 
        "//p[contains(@class, 'stock')]", "Stock details", book_details, "Stock status")

        if stocks_details:
            print(f"📦 Stock status: {stocks_details}")
            
    except Exception as e:
        print(f'Stock not found:\n{str(e)}')
    try:
        details = find_element(driver, "xpath", "//div[contains(@id, 'detallePrecio')]")
        sending_date = find_elements(details, "xpath", ".//strong[contains(@style, 'green')]")
        sending_date = f"{sending_date[0].text.strip()} - {sending_date[1].text.strip()}"
        book_details["sending_date"] = sending_date
        print(f"📦 Date d'envoi estimée : {sending_date}")
    except Exception as e:
        print(f"❌ Erreur sur la date d'envoi :\n{e}")

    try:
        book_description = find_element(details, "xpath", "//span[contains(@id,'texto-descripcion')]").text.strip()
        book_details["description"] = book_description
        print(f"📚 Description du livre : {book_description}")
    except Exception as e:
        print(f"❌ Erreur sur la description : {e}")
        
    try:
        author_description = find_element(driver, "xpath", ".//div[contains(@class, 'biography')]").text.strip()
        book_details["author_description"] = author_description
        print(f"👨‍🏫 Description de l'auteur : {author_description}")
    except Exception as e:
        print(f"❌ Erreur sur la description de l'auteur : {e}")

    percentages_by_star = get_star_percentage(driver)
    book_details["star_percentages"] = percentages_by_star
    driver.back()
    return book_details

In [ ]:
if page_results is not None:
    driver.get(page_results)
count_reached = 0
    
    print("📄 Page chargée avec succès.")
    Products_listings = find_element(driver, "xpath", "//div[contains(@class, 'productos')]")
    # Extraire les cartes de produits
    cards = find_elements(Products_listings, "xpath", "//div[contains(@class, 'box-producto')]")
    print(f"🛍 Nombre de produits trouvés : {len(cards)}")

    # Ouvrir le premier produit
    one_card = cards[0]
    card_href = one_card.find_element("xpath", ".//a").get_attribute('href')
    driver.get(card_href)
for i in range(4):
        wait_page_to_load(driver)
        get_book_details(driver)
        count_reached += 1
        

📄 Page chargée avec succès.
🛍 Nombre de produits trouvés : 50
📄 Page chargée avec succès.
Title: Nexus
Link: https://www.buscalibre.com.co/libro-nexus/9789566247593/p/62355533
Author: Noah Harari Yuval
Edition: Debate, 2024, Tapa Blanda, Nuevo
Price: $ 126.698
Old Price: $ 211.163

Discount: 40%
Going to product page...
📄 Page produit chargée avec succès.
📅 Année de publication: 2024
📊 Nombre d'avis : 13
Stock status: Quedan más de 100 unidades
📦 Stock status: Quedan más de 100 unidades
📦 Date d'envoi estimée : Lunes 19 de Mayo - Jueves 22 de Mayo
📚 Description du livre : Harari vuelve a contemplar a la humanidad bajo el prisma de la historia para mostrarnos hasta qué punto la información ha hecho y deshecho el mundo. Pese a los avances de los últimos cien mil años, nos enfrentamos a una gran crisis. En medio de la desinformación y el desconcierto, estamos al borde del colapso ecológico y nos precipitamos hacia la era de la Inteligencia Artificial. ¿Por qué somos una especie tan autode

{'Title': 'Nexus',
 'Link': 'https://www.buscalibre.com.co/libro-nexus/9789566247593/p/62355533',
 'Autor': 'Noah Harari Yuval',
 'Edition': 'Debate, 2024, Tapa Blanda, Nuevo',
 'Actual Price': '$ 126.698',
 'Old Price': '$ 211.163',
 'Discount': '40%',
 'year': '2024',
 'rating_amount': '13',
 'Stock details': 'Quedan más de 100 unidades',
 'sending_date': 'Lunes 19 de Mayo - Jueves 22 de Mayo',
 'description': 'Harari vuelve a contemplar a la humanidad bajo el prisma de la historia para mostrarnos hasta qué punto la información ha hecho y deshecho el mundo. Pese a los avances de los últimos cien mil años, nos enfrentamos a una gran crisis. En medio de la desinformación y el desconcierto, estamos al borde del colapso ecológico y nos precipitamos hacia la era de la Inteligencia Artificial. ¿Por qué somos una especie tan autodestructiva? A su más puro estilo, a partir de reveladores ejemplos históricos, ofrece un marco tremendamente útil para entender las complejas relaciones que existe

STAR PERCENTAGE TEST

In [ ]:
if page_results is not None:
    driver.get(page_results)

    wait_page_to_load(driver)
    print("📄 Page chargée avec succès.")

    # Extraire les cartes de produits
    cards = find_elements(driver, "xpath", "//div[contains(@class, 'box-producto')]")
    print(f"🛍 Nombre de produits trouvés : {len(cards)}")

    # Ouvrir le premier produit
    one_card = cards[0]
    card_href = one_card.find_element("xpath", ".//a").get_attribute('href')
    driver.get(card_href)


dict_star_pc = get_star_percentage(driver)
dict_star_pc

In [ ]:
if page_results is not None:
    driver.get(page_results)

    wait_page_to_load(driver)
    print("📄 Page chargée avec succès.")

    # Extraire les cartes de produits
    cards = find_elements(driver, "xpath", "//div[contains(@class, 'box-producto')]")
    print(f"🛍 Nombre de produits trouvés : {len(cards)}")

    # Ouvrir le premier produit
    one_card = cards[0]
    card_href = one_card.find_element("xpath", ".//a").get_attribute('href')
    driver.get(card_href)

star_percentages = {}  # Dictionnaire pour stocker les pourcentages par étoile

try:
    # Localiser l'élément contenant les répartitions d'avis par étoile
    evaluations_repartition = find_element(driver, "xpath", "//div[contains(@class, 'reviews-body-resume')]")
    
    # Obtenir tous les éléments <li> correspondant à chaque étoile (1 à 5 étoiles)
    each_star = find_elements(evaluations_repartition, "xpath", ".//li[contains(@class, 'stars-')]")
    
    # Parcourir chaque élément pour extraire le pourcentage par étoile
    for star in each_star:
        class_attr = star.get_attribute("class")
        match_star = re.search(r'stars-(\d)', class_attr)
        
        if match_star:
            star_rating = match_star.group(1)
            
            try:
                # Extraire le pourcentage du div avec la classe 'percent'
                percent_div = find_element(star, "xpath", ".//div[contains(@class, 'percent')]")
                style_attr = percent_div.get_attribute("style")
                
                # Extraire le pourcentage de la largeur du style CSS (e.g., "width: 85.71%")
                match_percentage = re.search(r'width:\s*(\d+(?:\.\d+)?)%', style_attr)
                
                if match_percentage:
                    percentage = match_percentage.group(1) + '%'  # Formater le pourcentage
                    star_percentages[star_rating] = percentage  # Ajouter au dictionnaire
                else:
                    print(f"❌ Pourcentage non trouvé pour {star_rating} étoiles")
            except Exception as e:
                print(f"❌ Erreur sur la récupération du pourcentage pour {star_rating} étoiles : {e}")
        else:
            print(f"❌ Classe d'étoile non trouvée pour {star}")
except Exception as e:
    print(f"❌ Erreur lors de l'extraction des pourcentages : {e}")



Collect Datas

In [ ]:
Products_listings = place_products_listings

if Products_listings:
    cards = find_elements(driver, "xpath",
    "//div[contains(@class, 'box-producto') and contains(@data-id_producto, '')]")
    if cards:
        print("Cards Found !")
        for card in cards:
            Products_listings = place_products_listings
            cards = find_elements(driver, "xpath",
    "//div[contains(@class, 'box-producto') and contains(@data-id_producto, '')]")

            title = card.find_element(By.XPATH, ".//h3[contains(@class, 'nombre')]").text.strip()  
            print(f"Title: {title}")

            href = card.find_element(By.XPATH, ".//a[contains(@href, '')]").get_attribute('href')
            print(f"Link: {href}")

            autor = find_element(card, "xpath", 
                ".//div[contains(@class, 'autor')]").text.strip()
            print(f"Author: {autor}")
            edition = find_element(card, "xpath", 
                ".//div[contains(@class, 'autor') and contains(@class,'color-dark-gray')]").text.strip()
            print(f"Edition: {edition}")
            stock = find_element(card, "xpath", ".//div[contains(@class, 'stock') and contains(@class,'green')]").text.strip()
            print(f"Stock status: {stock}")
            
            price = find_element(card, "xpath", 
                                        ".//p[contains(@class, 'precio-ahora')]").text.strip()
            print(f"Price: {price}")
            precio_antes = find_element(card, "xpath", 
                                        ".//p[contains(@class, 'precio-antes')]").text.strip()
            print(f"Old Price: {precio_antes}\n")